In [1]:
from pathlib import Path

import torch
torch.set_default_tensor_type(torch.cuda.FloatTensor)
torch.set_grad_enabled(False)

from tqdm.auto import tqdm

from lib.bayesn import get_cpdataset, get_ranges, load


N = 500
DATA_GEN = f'{N}-cosmo-ir'
GUIDE_KEY = 'hpmvn'
CONDITION_KEY = ''
SUFFIX = ''  # '-negrenyi'

EXPERIMENT_NAME = '-'.join((_ for _ in (GUIDE_KEY, CONDITION_KEY) if _))
SAVENAME = f'{EXPERIMENT_NAME}{SUFFIX}'
RESDIR = Path(f'res-{DATA_GEN}')

NNTRAIN = RESDIR / 'nntrain' / f'{SAVENAME}-nntrain-{{i}}.pt'


guidename = RESDIR / f'{SAVENAME}-guide.pt'

config = load(EXPERIMENT_NAME, DATA_GEN, N, guidename=guidename)
config.guide.eval()

Guide(
  (hpmvn): HierarchicPartialMultivariateNormalSamplingGroup(8 sites, torch.Size([1505]))
)

In [2]:
ranges = get_ranges(config, ignore=('Om0', 'Ode0', 'delta_M'), nsigma=3)
dataset = get_cpdataset(config, CONDITION_KEY, ranges, (0.1, 0.5, 0.5, 0.9))

In [3]:
from more_itertools import take

for i in range(7):
    torch.save(
        take(N := 3000, tqdm(dataset, total=N)), NNTRAIN.format(i=i)
    )

  0%|          | 0/3000 [00:00<?, ?it/s]

/home/kosio/Projects/Python/pyrofit-utils/pyrofit/utils/interpolate/abc.py:34: UserWarning: input value tensor is non-contiguous, this will lower the performance due to extra data copy when converting non-contiguous tensor to contiguous, please use contiguous input value tensor if possible (Triggered internally at  /build/python-pytorch/src/pytorch-1.9.1-opt-cuda/aten/src/ATen/native/BucketizationUtils.h:19.)
  return searchsorted(*broadcast_except(grid, x, dim=-1), right=True).clamp_(1, grid.shape[-1]-1).sub_(1)


KeyboardInterrupt: 